In [82]:
import os,cv2
import time
import numpy as np
import matplotlib.pyplot as plt

from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split

# from keras import backend as K

# K.set_image_dim_ordering('th')

from keras.utils import np_utils
import keras
# from keras.layers.core import Reshape
# from keras.models import Sequential
# from keras.layers.core import Dense, Dropout, Activation, Flatten
# from keras.layers.convolutional import Convolution2D, MaxPooling2D
# from keras.optimizers import SGD,RMSprop,adam

from keras.models import Sequential
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers.core import Activation
from keras.layers.core import Flatten
from keras.layers.core import Dense
from keras.layers.core import Dropout
from keras import backend as K
K.set_image_dim_ordering('tf')
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator

In [83]:
img_rows=120
img_cols=240
num_channel=1
num_epoch=20

# Define the number of classes
num_classes = 3

img_data_list=[]

In [84]:
PATH = os.getcwd()
# Define data path
data_path = PATH + '/IOT_Project/training_images_raw'
data_dir_list = os.listdir(data_path)
count = 1

In [85]:
for datadir in data_dir_list:
    #print("Processing : " + str(datadir))
    if not (datadir.endswith('.DS_Store') or datadir.endswith("l") or datadir.endswith("r")):
        images_in_dir = os.listdir(data_path + '/' + datadir)
        print("Size of " + str(datadir) + " : " + str(len(images_in_dir)))
        for single_image in images_in_dir:
            if single_image.endswith('.DS_Store'):
                print("found 1 ds in " + datadir)
            else:    
                input_img = cv2.imread(data_path + '/'+ datadir + '/'+ single_image, 0)
                input_img = input_img.reshape((1, 38400))
                img_data_list.append(input_img)
                    

Size of fwd : 207
found 1 ds in fwd
Size of fwd_left : 148
found 1 ds in fwd_left
Size of fwd_right : 102
found 1 ds in fwd_right


In [86]:
len(img_data_list)

454

In [87]:
img_data = np.array(img_data_list)
img_data = img_data.astype('float32')
img_data /= 255
print (img_data.shape)

(454, 1, 38400)


In [48]:
#img_data = np.expand_dims(img_data, axis=4) 
#print (img_data.shape)

(454, 120, 320, 1)


/Users/sumitthorat/Documents/self_driving_car/venv/lib/python2.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Both axis > a.ndim and axis < -a.ndim - 1 are deprecated and will raise an AxisError in the future.
  """Entry point for launching an IPython kernel.


In [49]:
num_classes = 3

In [50]:
num_of_samples = img_data.shape[0]
labels = np.ones((num_of_samples,),dtype='int64')

In [51]:
labels[0:206]=0
labels[206:353]=1
labels[353:454]=2
# labels[423:238]=3
# labels[238:] = 4

names = ['fwd','fwd_left','fwd_right']

Y = np_utils.to_categorical(labels, num_classes)



In [52]:
x,y = shuffle(img_data,Y, random_state=2)

In [53]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=2)

In [54]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(363, 120, 320, 1)
(91, 120, 320, 1)
(363, 3)
(91, 3)


In [55]:
inputShape=img_data[0].shape

In [56]:
aug = ImageDataGenerator(rotation_range=5, width_shift_range=0.1, height_shift_range=0.1, shear_range=0.0, zoom_range=0.1,horizontal_flip=False, fill_mode="nearest")

In [57]:
model = Sequential()
inputShape = (120, 320, 1)

model.add(Dense(units=128,activation="relu",input_shape=(38400,)))
model.add(Dense(units=128,activation="relu"))
model.add(Dense(units=128,activation="relu"))
model.add(Dense(units=3,activation="softmax"))







In [58]:
EPOCHS = 10
INIT_LR = 1e-3
BS = 32

opt = Adam(lr=INIT_LR, decay=INIT_LR / EPOCHS)
model.compile(loss="categorical_crossentropy", optimizer=opt, metrics=["accuracy"])

In [59]:
#hist = model.fit(X_train, y_train, batch_size=16, epochs=20, verbose=1, validation_data=(X_test, y_test))
H = model.fit_generator(aug.flow(X_train, y_train, batch_size=BS), validation_data=(X_test, y_test), steps_per_epoch=len(X_train) // BS,epochs=EPOCHS, verbose=1)

Epoch 1/10


 1/11 [=>............................] - ETA: 19s - loss: 1.0941 - acc: 0.4375

 2/11 [====>.........................] - ETA: 14s - loss: 2.2584 - acc: 0.4531

 3/11 [=======>......................] - ETA: 11s - loss: 1.9807 - acc: 0.3958

 4/11 [=========>....................] - ETA: 9s - loss: 1.7883 - acc: 0.3672 

...

 7/11 [==================>...........] - ETA: 6s - loss: 1.0461 - acc: 0.4359

 8/11 [====================>.........] - ETA: 5s - loss: 1.0354 - acc: 0.4556

 9/11 [=======================>......] - ETA: 3s - loss: 1.0293 - acc: 0.4605

10/11 [==========================>...] - ETA: 1s - loss: 1.0406 - acc: 0.4509

11/11 [==============================] - 20s 2s/step - loss: 1.0351 - acc: 0.4477 - val_loss: 0.9203 - val_acc: 0.5714


In [64]:
score = model.evaluate(X_test, y_test, verbose=0)
print('Test Loss:', score[0])
print('Test accuracy:', score[1])

('Test Loss:', 0.9202611747678819)
('Test accuracy:', 0.571428568153591)


In [67]:
model.save("test_model3.h5")

ImportError: `save_model` requires h5py.

In [76]:
test_image = cv2.imread("IOT_Project/test_image_tape.jpg", 0)

In [77]:
test_image = np.array(test_image)
test_image = test_image.astype('float32')
test_image /= 255
print(test_image)

[[0.4745098  0.47843137 0.47843137 ... 0.45490196 0.45490196 0.39607844]
 [0.47058824 0.4745098  0.47843137 ... 0.45490196 0.45490196 0.4       ]
 [0.46666667 0.4745098  0.4745098  ... 0.44705883 0.4509804  0.39607844]
 ...
 [0.44705883 0.40392157 0.4117647  ... 0.40784314 0.41960785 0.3647059 ]
 [0.4        0.40392157 0.4117647  ... 0.4117647  0.41960785 0.36078432]
 [0.40392157 0.41568628 0.40784314 ... 0.41568628 0.41960785 0.35686275]]


In [78]:
test_image= np.expand_dims(test_image, axis=3) 
test_image= np.expand_dims(test_image, axis=0)
print (test_image.shape)

(1, 120, 320, 1)


/Users/sumitthorat/Documents/self_driving_car/venv/lib/python2.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Both axis > a.ndim and axis < -a.ndim - 1 are deprecated and will raise an AxisError in the future.
  """Entry point for launching an IPython kernel.


In [79]:
prediction = model.predict(test_image)

In [80]:
prediction

array([[0.46256077, 0.3430989 , 0.19434033]], dtype=float32)

In [81]:
print(model.predict_classes(test_image))

[0]


In [128]:
print(y_test[0:1])

[[0. 1. 0. 0. 0.]]
